In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset, SwatItrustDataset
from utils.custom_plots import plot_stacked
from utils.metrics import time_span_metrics
from utils.watchmen import LimitWatchman, LimitPcaWatchman, SpePcaWatchman, IsolatingWatchman, LinearPredictWatchman

In [4]:
SEED = 1729

# Preparing datasets

In [5]:
datasets = {
    0: GhlKasperskyDataset(),
    1: TepHarvardDataset(),
    2: TepKasperskyDataset(),
    3: SwatItrustDataset(),
}

In [6]:
shake_kwargs = {
    'random_state': SEED,
    'valid_test_ratio': 0.3,
}

# Preparing watchmen

In [7]:
watchmen = {
    0: {  # GhlKasperskyDataset
        0: LimitWatchman(),
        1: LimitPcaWatchman(n_components=3),
        2: SpePcaWatchman(n_components=3),
        3: IsolatingWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
    },
    1: {  # TepHarvardDataset
        0: LimitWatchman(),  # better: ewma='3 min'
        1: LimitPcaWatchman(n_components=3),  # better: n_components=12
        2: SpePcaWatchman(n_components=3),  # better: n_components=12
        3: IsolatingWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
    },
    2: {  # TepKasperskyDataset
        0: LimitWatchman(),
        1: LimitPcaWatchman(n_components=3),
        2: SpePcaWatchman(n_components=3),
        3: IsolatingWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
    },
    3: {  # SwatItrustDataset
        0: LimitWatchman(),
        1: LimitPcaWatchman(n_components=3),
        2: SpePcaWatchman(n_components=3),
        3: IsolatingWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
    },
}

# Learning

## Prefit

In [8]:
for d in datasets:
    datasets[d].shake_not_stir(**shake_kwargs)
    for data, _, _ in tqdm(datasets[d].train_generator(), desc=f'Prefit on train {d}'):
        for w in watchmen[d]:
            watchmen[d][w].prefit(data)

Prefit on train 0: 0it [00:00, ?it/s]

Prefit on train 3: 0it [00:00, ?it/s]

## Fit

In [9]:
for d in datasets:
    datasets[d].shake_not_stir(**shake_kwargs)
    for data, _, _ in tqdm(datasets[d].train_generator(), desc=f'Fit on train {d}'):
        for w in watchmen[d]:
            watchmen[d][w].partial_fit(data)

Fit on train 0: 0it [00:00, ?it/s]

Fit on train 3: 0it [00:00, ?it/s]

# Examine

## Throw stones

In [10]:
stones = dict()
for d in datasets:
    stones[d] = dict()
    for w in watchmen[d]:
        stones[d][w] = list()

In [11]:
for d in datasets:
    datasets[d].shake_not_stir(**shake_kwargs)
    for data, faults, info in tqdm(datasets[d].valid_generator(), desc=f'Detect on validation {d}'):
        for w in watchmen[d]:
            detect = watchmen[d][w].predict(data)
            stones[d][w].append(pd.concat([faults, detect], axis=1))
            stones[d][w][-1].index.name = info

Detect on validation 0: 0it [00:00, ?it/s]

Detect on validation 3: 0it [00:00, ?it/s]

## Individual results

In [12]:
metrics = ('precision', 'recall', 'f1_score')
indi_results = dict()
for d in datasets:
    indi_results[str(datasets[d])] = pd.DataFrame(columns=metrics)
    for w in watchmen[d]:
        exam_paper = pd.DataFrame(columns=metrics)
        for st in stones[d][w]:
            exam_paper.loc[st.index.name, metrics] = time_span_metrics(st.iloc[:, 0], st.iloc[:, 1:])
        indi_results[str(datasets[d])].loc[str(watchmen[d][w]), metrics] = exam_paper.mean().values

In [13]:
for d in indi_results:
    print(d)
    display(indi_results[d])

GhlKasperskyDataset(E:\Datasets\GHL)


,precision,recall,f1_score
LimitWatchman(ewma=None),0.207358,0.638095,0.281676
LimitPcaWatchman(n_components=3),0.284864,0.328571,0.298718
SpePcaWatchman(n_components=3),0.0,0.0,0.0
IsolatingWatchman(n_trees=17),0.017045,1.0,0.033429
LinearPredictWatchman(n_features=12),0.014094,1.0,0.02773


SwatItrustDataset(E:\Datasets\SWaT\dataset12)


,precision,recall,f1_score
LimitWatchman(ewma=None),0.226415,1.0,0.369231
LimitPcaWatchman(n_components=3),NaN,0.0,0.0
SpePcaWatchman(n_components=3),NaN,0.0,0.0
IsolatingWatchman(n_trees=6),0.222222,1.0,0.363636
LinearPredictWatchman(n_features=51),0.375,0.25,0.3


## Ensembling results

### Threshold
Watchman can detect anomaly by several features.

In [50]:
metrics = ('precision', 'recall', 'f1_score')
union_results = dict()
max_threshold = 10
for d in datasets:
    union_results[str(datasets[d])] = pd.DataFrame(columns=metrics)
    exam_papers = dict()
    for threshold in range(1, max_threshold+1):
        exam_papers[threshold] = pd.DataFrame(columns=metrics)
    for i_st, st in enumerate(stones[d][0]):
        opinions = pd.concat([stones[d][w][i_st].iloc[:, 1:].sum(axis=1) for w in watchmen[d]], axis=1)
        for threshold in range(1, max_threshold+1):
            detect = (opinions.sum(axis=1) >= threshold).astype('uint8')
            exam_papers[threshold].loc[i_st, metrics] = time_span_metrics(st.iloc[:, 0], detect)
    for threshold in range(1, max_threshold+1):
        union_results[str(datasets[d])].loc[threshold, metrics] = exam_papers[threshold].mean().values

In [51]:
for d in union_results:
    print(d)
    display(union_results[d])

GhlKasperskyDataset(E:\Datasets\GHL)


,precision,recall,f1_score
1,0.014094,1.0,0.02773
2,0.016777,1.0,0.032914
3,0.187554,0.666667,0.275014
4,0.292491,0.3,0.278211
5,NaN,0.0,0.0
6,NaN,0.0,0.0
7,NaN,0.0,0.0
8,NaN,0.0,0.0
9,NaN,0.0,0.0
10,NaN,0.0,0.0


SwatItrustDataset(E:\Datasets\SWaT\dataset12)


,precision,recall,f1_score
1,0.164303,1.0,0.277972
2,0.184636,0.95,0.307903
3,0.361111,0.416667,0.383333
4,0.666667,0.275,0.373626
5,0.714286,0.225,0.324561
6,1.0,0.183333,0.309524
7,1.0,0.091667,0.167832
8,1.0,0.091667,0.167832
9,1.0,0.091667,0.167832
10,1.0,0.091667,0.167832


### Probability
Something like probabilty for every watchman.

In [56]:
metrics = ('precision', 'recall', 'f1_score')
union_results = dict()
for d in datasets:
    union_results[str(datasets[d])] = pd.DataFrame(columns=metrics)
    exam_papers = dict()
    for threshold in range(5, 51, 5):
        exam_papers[threshold] = pd.DataFrame(columns=metrics)
    for i_st, st in enumerate(stones[d][0]):
        opinions = pd.concat([stones[d][w][i_st].iloc[:, 1:].mean(axis=1) for w in watchmen[d]], axis=1)
        for threshold in range(5, 51, 5):
            detect = (opinions.sum(axis=1) >= threshold*0.01).astype('uint8')
            exam_papers[threshold].loc[i_st, metrics] = time_span_metrics(st.iloc[:, 0], detect)
    for threshold in range(5, 51, 5):
        union_results[str(datasets[d])].loc[threshold*0.01, metrics] = exam_papers[threshold].mean().values

In [57]:
for d in union_results:
    print(d)
    display(union_results[d])

GhlKasperskyDataset(E:\Datasets\GHL)


,precision,recall,f1_score
0.05,0.014094,1.0,0.02773
0.10,0.016777,1.0,0.032914
0.15,0.016777,1.0,0.032914
0.20,0.016954,1.0,0.033254
0.25,0.016998,1.0,0.03334
0.30,0.016998,1.0,0.03334
0.35,0.016998,1.0,0.03334
0.40,0.016998,1.0,0.03334
0.45,0.01704,1.0,0.03342
0.50,0.017045,1.0,0.033429


SwatItrustDataset(E:\Datasets\SWaT\dataset12)


,precision,recall,f1_score
0.05,0.164303,1.0,0.277972
0.10,0.164303,1.0,0.277972
0.15,0.164303,1.0,0.277972
0.20,0.164303,1.0,0.277972
0.25,0.164303,1.0,0.277972
0.30,0.164303,1.0,0.277972
0.35,0.164303,1.0,0.277972
0.40,0.164303,1.0,0.277972
0.45,0.164303,1.0,0.277972
0.50,0.164303,1.0,0.277972
